In [2]:
import requests
import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
# import TwitterUtils as TU
import seaborn as sns
import re

# import spacy
# import spacy_langdetect as sld

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")

def generate(instruction, knowledge, dialog):
    if knowledge != '':
        knowledge = '[KNOWLEDGE] ' + knowledge
    dialog = ' EOS '.join(dialog)
    query = f"{instruction} [CONTEXT] {dialog} {knowledge}"
    input_ids = tokenizer(f"{query}", return_tensors="pt").input_ids
    outputs = model.generate(input_ids, max_length=128, min_length=8, top_p=0.9, do_sample=True)
    output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return output

# Instruction for a chitchat task
instruction = f'Instruction: given a dialog context, you need to response empathically.'
# Leave the knowldge empty
knowledge = ''
dialog = [
    'Does money buy happiness?',
    'It is a question. Money buys you a lot of things, but not enough to buy happiness.',
    'What is the best way to buy happiness ?'
]
response = generate(instruction, knowledge, dialog)
print(response)

/Users/stephentoner/miniconda3/envs/SI699proj/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


money doesn't buy happiness.


In [4]:
convo_df = pd.read_csv("convo_data.csv")

In [5]:
convo_df.iloc[42].clean_convo

'[1] https://t.co/xRm5O7VaCt|| [2] Respect my goat \U0001fae1 https://t.co/qtWbliK8Nt|| [0] Tatum sucks|| [3] Respect yourself|| [1] https://t.co/cr47l4df9P|| [2] 😭||'

In [6]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

dialog = preprocess(convo_df.iloc[60].clean_convo).split('||')




In [7]:
dialog

['[3] Not bad!',
 " [6] I'm wondering how much it would cost to replicate the build today.",
 ' [4] Hah, I’m almost surprised that didn’t go for slightly more, tbh.',
 ' [5] $4800 lmao',
 " [2] Fairly significant car. $100k wouldn't have surprised me at all.",
 ' [0] 10-15k all day.',
 ' [6] Check the specs  https://t.co/uk03t8K1ih',
 " [1] It was Paul Newman's car its a legendary volvo that belonged to a legendary man",
 '']

In [8]:
# Instruction for a chitchat task
instruction = f'Instruction: given a dialog context, respond in the style of the dialog.'
# Leave the knowldge empty
knowledge = ''

response = generate(instruction, knowledge, dialog)
print(response)

[1] It's a legendary volvo that belonged to a legendary man
